# Imports

In [53]:
import numpy as np
import pandas as pd
import random
import math
from collections import Counter, defaultdict
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Split data

In [55]:
def create_split(X, y, random_state=42):
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.30, stratify=y, random_state=random_state
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=random_state
    )
    return (X_train.reset_index(drop=True), y_train.reset_index(drop=True)), \
           (X_val.reset_index(drop=True), y_val.reset_index(drop=True)), \
           (X_test.reset_index(drop=True), y_test.reset_index(drop=True))

# Core functions for Decision Tree


In [57]:
def entropy(labels):
    if len(labels) == 0: return 0.0
    counts = Counter(labels)
    n = len(labels)
    ent = 0.0
    for count in counts.values():
        p = count / n
        if p > 0:
            ent -= p * np.log2(p)
    return ent

def information_gain(parent_labels, splits):
    H_parent = entropy(parent_labels)
    n = len(parent_labels)
    weighted_child_entropy = 0.0
    for child_labels in splits.values():
        if len(child_labels) > 0:
            weighted_child_entropy += (len(child_labels)/n) * entropy(child_labels)
    return H_parent - weighted_child_entropy

def find_best_split_continuous(X_col, y, min_samples_leaf):
    data = pd.DataFrame({'feature': X_col, 'label': y}).sort_values('feature')
    unique_values = data['feature'].unique()
    if len(unique_values)<2: return None, None, None
    candidate_thresholds = (unique_values[:-1]+unique_values[1:])/2
    best_gain, best_threshold, best_splits = -1, None, None
    n = len(data)
    for threshold in candidate_thresholds:
        left_mask = data['feature'] <= threshold
        y_left = data['label'].loc[left_mask.index[left_mask]]
        y_right = data['label'].loc[left_mask.index[~left_mask]]
        if len(y_left)<min_samples_leaf or len(y_right)<min_samples_leaf: continue
        splits = {f'<={threshold:.4f}': y_left, f'>{threshold:.4f}': y_right}
        gain = information_gain(y, splits)
        if gain > best_gain:
            best_gain, best_threshold, best_splits = gain, threshold, splits
    return best_threshold, best_gain, best_splits

# ----------------------------
# 2) Node & DecisionTreeIG
# ----------------------------
class Node:
    def __init__(self, depth=0):
        self.depth = depth
        self.is_leaf = False
        self.prediction = None
        self.feature = None
        self.split_value = None
        self.children = {}
        self.n_samples = 0
        self.n_classes = {}
        self.entropy = 0.0
        self.is_continuous_split = False

class DecisionTreeIG:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None
        self.feature_importance_ = defaultdict(float)
        self.n_nodes_ = 0
        self.max_observed_depth_ = 0
        self.feature_gains_ = defaultdict(list)

    def fit(self, X, y):
        self.feature_types = {col:X[col].dtype for col in X.columns}
        self.root = self._build_tree(X, y, list(X.columns), 0)
        total_importance = sum(self.feature_importance_.values())
        if total_importance>0:
            for feat in self.feature_importance_: self.feature_importance_[feat]/=total_importance
        return self

    def _majority_class(self, labels):
        return Counter(labels).most_common(1)[0][0]

    def _build_tree(self, X, y, available_features, depth):
        node = Node(depth=depth)
        node.n_samples = len(y)
        node.n_classes = dict(Counter(y))
        node.entropy = entropy(y)
        self.n_nodes_ += 1
        self.max_observed_depth_ = max(self.max_observed_depth_, depth)
        if len(set(y))==1 or not available_features or (self.max_depth and depth>=self.max_depth) or len(y)<self.min_samples_split:
            node.is_leaf = True
            node.prediction = self._majority_class(y)
            return node
        best_feature, best_gain, best_splits, best_split_value, is_continuous = None, -1, None, None, False
        for feature in available_features:
            f_dtype = self.feature_types[feature]
            if f_dtype in ['float64','int64']:
                threshold, gain, splits = find_best_split_continuous(X[feature], y, self.min_samples_leaf)
                if gain is not None and gain>best_gain:
                    best_feature, best_gain, best_splits, best_split_value, is_continuous = feature, gain, splits, threshold, True
            else:
                groups = {v:y.loc[idxs] for v, idxs in X.groupby(feature).groups.items()}
                if any(len(lbls)<self.min_samples_leaf for lbls in groups.values()): continue
                gain = information_gain(y, groups)
                if gain>best_gain: best_feature, best_gain, best_splits, best_split_value, is_continuous = feature, gain, groups, None, False
        if best_feature is None or best_gain<=1e-12:
            node.is_leaf = True
            node.prediction = self._majority_class(y)
            return node
        self.feature_importance_[best_feature] += best_gain
        self.feature_gains_[best_feature].append(best_gain)
        node.feature = best_feature
        node.split_value = best_split_value
        node.is_continuous_split = is_continuous
        remaining_features = available_features.copy()
        if is_continuous:
            threshold = best_split_value
            mask_left = X[best_feature]<=threshold
            node.children[f'<={threshold:.4f}'] = self._build_tree(X.loc[mask_left,remaining_features], y.loc[mask_left], remaining_features, depth+1)
            node.children[f'>{threshold:.4f}'] = self._build_tree(X.loc[~mask_left,remaining_features], y.loc[~mask_left], remaining_features, depth+1)
        else:
            for value, lbls in best_splits.items():
                mask = X[best_feature]==value
                node.children[value] = self._build_tree(X.loc[mask,remaining_features], y.loc[mask], remaining_features, depth+1)
        node.prediction = self._majority_class(y)
        return node

    def predict_one(self, x):
        node = self.root
        while not node.is_leaf:
            val = x.get(node.feature, 'UNKNOWN')
            if node.is_continuous_split:
                if val=='UNKNOWN': return node.prediction
                branch = f'<={node.split_value:.4f}' if val<=node.split_value else f'>{node.split_value:.4f}'
                node = node.children.get(branch, Node()) 
            else:
                node = node.children.get(val, Node())
            if node is None: return None
        return node.prediction

    def predict(self, X):
        return np.array([self.predict_one(X.iloc[i].to_dict()) for i in range(len(X))])


# DecisionTreeIG subclass for feature subspace (for Random Forest)


In [59]:
class DecisionTreeIG_Subspace(DecisionTreeIG):
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1,
                 max_features_at_split=None, random_state=None):
        super().__init__(max_depth, min_samples_split, min_samples_leaf)
        self.max_features_at_split = max_features_at_split
        if random_state: random.seed(random_state); np.random.seed(random_state)

    def _build_tree(self, X, y, available_features, depth):
        if self.max_features_at_split is not None and len(available_features)>self.max_features_at_split:
            available_features = random.sample(available_features, self.max_features_at_split)
        return super()._build_tree(X, y, available_features, depth)

# Random Forest class


In [61]:
class RandomForest_Subspace:
    def __init__(self, n_estimators=10, max_features=None, max_depth=None,
                 min_samples_split=2, min_samples_leaf=1, random_state=None, verbose=False):
        self.n_estimators, self.max_features = n_estimators, max_features
        self.max_depth, self.min_samples_split, self.min_samples_leaf = max_depth, min_samples_split, min_samples_leaf
        self.random_state, self.trees, self.verbose = random_state, [], verbose
        if random_state: random.seed(random_state); np.random.seed(random_state)

    def fit(self,X,y):
        n = len(X)
        self.trees=[]
        for i in range(self.n_estimators):
            idxs = np.random.choice(n,n,replace=True)
            Xb, yb = X.iloc[idxs].reset_index(drop=True), y.iloc[idxs].reset_index(drop=True)
            tree = DecisionTreeIG_Subspace(max_depth=self.max_depth, min_samples_split=self.min_samples_split,
                                           min_samples_leaf=self.min_samples_leaf, max_features_at_split=self.max_features,
                                           random_state=None if self.random_state is None else self.random_state+i)
            tree.fit(Xb,yb)
            self.trees.append(tree)
            if self.verbose: print(f"Tree {i+1}/{self.n_estimators} trained, nodes={tree.n_nodes_}")
        return self

    def predict(self,X):
        preds = np.array([tree.predict(X) for tree in self.trees])
        return np.array([Counter(row).most_common(1)[0][0] for row in preds.T])


# Load dataset & split


In [63]:
data = load_breast_cancer(as_frame=True)
df = data.frame.copy()
df['class'] = df['target'].map({0:'malignant',1:'benign'})
df = df.drop(columns=['target'])
X_full = df.drop(columns=['class'])
y_full = df['class']
(X_train, y_train), (X_val, y_val), (X_test, y_test) = create_split(X_full, y_full)


# Random Forest tuning & evaluation


In [78]:
d = X_train.shape[1]
mf_options = sorted(list(set([int(math.floor(math.sqrt(d))), int(math.floor(d/2))])))
T_values = [5,10,30,50]
depth_choice, min_split_choice = 6, 2

best_rf, best_cfg, best_acc = None, None, -1
results=[]
for T in T_values:
    for mf in mf_options:
        rf = RandomForest_Subspace(n_estimators=T, max_features=mf, max_depth=depth_choice,
                                   min_samples_split=min_split_choice, min_samples_leaf=1, verbose=False)
        rf.fit(X_train,y_train)
        val_acc = accuracy_score(y_val, rf.predict(X_val))
        results.append({'T':T,'max_features':mf,'val_acc':val_acc})
        if val_acc>best_acc: best_acc,best_cfg,best_rf=val_acc,(T,mf),rf

results_df = pd.DataFrame(results).sort_values('val_acc',ascending=False).reset_index(drop=True)
print("\nTop tuning results:\n", results_df.head(10))
print(f"\nBest config: T={best_cfg[0]}, max_features={best_cfg[1]} -> val_acc={best_acc:.4f}")



Top tuning results:
     T  max_features   val_acc
0  10            15  0.964706
1  30            15  0.952941
2  30             5  0.941176
3  50            15  0.941176
4   5             5  0.929412
5   5            15  0.929412
6  50             5  0.929412
7  10             5  0.917647

Best config: T=10, max_features=15 -> val_acc=0.9647


# Retrain best RF on train+val, evaluate test


In [79]:
X_train_val = pd.concat([X_train,X_val],axis=0).reset_index(drop=True)
y_train_val = pd.concat([y_train,y_val],axis=0).reset_index(drop=True)
best_T, best_mf = best_cfg
rf_final = RandomForest_Subspace(n_estimators=best_T,max_features=best_mf,
                                 max_depth=depth_choice,min_samples_split=min_split_choice,
                                 random_state=42,verbose=True)
rf_final.fit(X_train_val,y_train_val)
y_test_pred_rf = rf_final.predict(X_test)
rf_test_acc = accuracy_score(y_test,y_test_pred_rf)
print(f"\nRandom Forest Test Accuracy: {rf_test_acc:.4f}")
print(classification_report(y_test,y_test_pred_rf,digits=4))

Tree 1/10 trained, nodes=23
Tree 2/10 trained, nodes=25
Tree 3/10 trained, nodes=27
Tree 4/10 trained, nodes=23
Tree 5/10 trained, nodes=25
Tree 6/10 trained, nodes=27
Tree 7/10 trained, nodes=23
Tree 8/10 trained, nodes=29
Tree 9/10 trained, nodes=21
Tree 10/10 trained, nodes=23

Random Forest Test Accuracy: 0.9767
              precision    recall  f1-score   support

      benign     0.9643    1.0000    0.9818        54
   malignant     1.0000    0.9375    0.9677        32

    accuracy                         0.9767        86
   macro avg     0.9821    0.9688    0.9748        86
weighted avg     0.9776    0.9767    0.9766        86

